# 5. New York car crash medical response

Imagine that you are part of a crisis response team, and you want to identify how hospitals have been responding to crash collisions in New York City.

We will use two data set:
1. crash event in New York city
2. hospitals location in New York city


In [1]:
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon

import folium
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster

In [2]:
# this function can help you to visualize your map correctly
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    # return IFrame(file_name, width='100%', height='500px')

## Exercises 1. Visualize the collision data.

Run the code cell below to load a GeoDataFrame `collisions` which contains all major motor vehicle collisions in 2013-2018 of New York city.

In [3]:
root_path="/home/pliu/data_set/kaggle/geospatial/L05"
collision_file_path=f"{root_path}/NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions.shp"

In [4]:
collision=gpd.read_file(collision_file_path)
collision.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,...,CONTRIBU_2,CONTRIBU_3,CONTRIBU_4,UNIQUE KEY,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry
0,07/30/2019,0:00,BRONX,10464,40.841100,-73.784960,"(40.8411, -73.78496)",None,None,121 PILOT STREET,...,Unspecified,None,None,4180045,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,POINT (1043750.211 245785.815)
1,07/30/2019,0:10,QUEENS,11423,40.710827,-73.770660,"(40.710827, -73.77066)",JAMAICA AVENUE,188 STREET,None,...,None,None,None,4180007,Sedan,Sedan,None,None,None,POINT (1047831.185 198333.171)
2,07/30/2019,0:25,None,None,40.880318,-73.841286,"(40.880318, -73.841286)",BOSTON ROAD,None,None,...,None,None,None,4179575,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (1028139.293 260041.178)
3,07/30/2019,0:35,MANHATTAN,10036,40.756744,-73.984590,"(40.756744, -73.98459)",None,None,155 WEST 44 STREET,...,None,None,None,4179544,Box Truck,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (988519.261 214979.320)
4,07/30/2019,10:00,BROOKLYN,11223,40.600090,-73.965910,"(40.60009, -73.96591)",AVENUE T,OCEAN PARKWAY,None,...,None,None,None,4180660,Station Wagon/Sport Utility Vehicle,Bike,None,None,None,POINT (993716.669 157907.212)


Use the `LATITUDE` and `LONGITUDE` columns to create an interactive map to visualize the collision data.  **What type of map do you think is most effective?**

In [ ]:
m_1 = folium.Map(location=[40.7, -74], tiles='cartodbpositron',zoom_start=11)

# Your code here: Visualize the collision data
for idx,row in collision.iterrows():
    Marker([row['LATITUDE'], row['LONGITUDE']]).add_to(m_1)

# Show the map
embed_map(m_1,"colliion.html")


### 2) Understand hospital coverage.

Below code cell to load the hospital data.